In [2]:
import os
plt.xlabel('k')
plt.ylabel('Silhouette')
plt.title('Silhouette vs k')
plt.tight_layout()
plt.savefig('k_selection.png', dpi=150)
plt.close()


best_idx = int(np.nanargmax(silhouettes))
best_k = ks[best_idx]
print('Mejor k según silhouette:', best_k)


kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=30)
cluster_labels = kmeans.fit_predict(X_processed)


# Guardar modelo
joblib.dump(kmeans, 'kmeans_train.joblib')


# Crear dataframe de salida con cluster y target
out = pd.DataFrame({'id': ids.values, 'cluster': cluster_labels, 'default': y.values})
out.to_csv('clusters_train.csv', index=False)


# ---------------------------
# 6) Resumen por cluster
# ---------------------------
summary = out.groupby('cluster').agg(
size=('id','count'),
default_rate=('default','mean')
).reset_index().sort_values('cluster')


# Añadir medias de variables numéricas por cluster (usar X original para interpretabilidad)
X_num = X.reset_index(drop=True)
X_num['cluster'] = cluster_labels
num_stats = X_num.groupby('cluster')[num_cols].mean().reset_index()
cluster_summary = summary.merge(num_stats, on='cluster')
cluster_summary.to_csv('cluster_summary.csv', index=False)
print('Cluster summary:)
', cluster_summary[['cluster','size','default_rate']])


# ---------------------------
# 7) Visualizar clusters sobre PCA2
# ---------------------------
plt.figure(figsize=(8,6))
palette = sns.color_palette('tab10', n_colors=best_k)
for cl in range(best_k):
sel = cluster_labels == cl
plt.scatter(X_pca2[sel,0], X_pca2[sel,1], s=15, alpha=0.7, label=f'Cl {cl}')
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clusters K-Means sobre PCA(2)')
plt.tight_layout()
plt.savefig('clusters_pca2.png', dpi=150)
plt.close()


# ---------------------------
# 8) Si existen predicciones supervisadas, correlacionar
# ---------------------------
if os.path.exists('supervised_train_preds.csv'):
sp = pd.read_csv('supervised_train_preds.csv')
if 'id' in sp.columns and 'pred_prob' in sp.columns:
merged = out.merge(sp, on='id')
corr = merged.groupby('cluster').agg(
mean_pred_prob=('pred_prob','mean'),
default_rate=('default','mean'),
size=('id','count')
).reset_index()
corr.to_csv('cluster_vs_predprob.csv', index=False)
print('Generado cluster_vs_predprob.csv con correlación entre predicciones y default por cluster')
else:
print('Archivo supervised_train_preds.csv encontrado pero faltan las columnas `id` o `pred_prob`.')
else:
print('No se encontró supervised_train_preds.csv — se omite comparación con modelo supervisado.')


# ---------------------------
# 9) Output final y recomendaciones
# ---------------------------
print('\nOutputs generados:')
for f in ['preprocessor_train.joblib','kmeans_train.joblib','clusters_train.csv','cluster_summary.csv','pca2_overview.png','k_selection.png','clusters_pca2.png']:
if os.path.exists(f):
print(' -', f)


print('\nRecomendaciones operativas (para incluir en README):')
print(' - Usar clusters para caracterizar subpoblaciones y ajustar umbrales de score por segmento.')
print(' - Reentrenar/validar el clasificador por segmento si se detectan diferencias importantes en feature importance o rendimiento.')
print(' - Investigar clusters con alta tasa de default: diseñar reglas de negocio o pruebas de verificación adicionales.')
print(' - Tratar outliers por separado: analizarlos con métodos de detección de anomalías (IsolationForest) antes de excluirlos automáticamente.')


# Guardar resumen breve a un archivo txt
with open('README_summary.txt','w',encoding='utf-8') as f:
f.write('K-Means + PCA aplicado sobre train.csv. Outputs: clusters_train.csv, cluster_summary.csv, modelos joblib y figuras.\n')


print('\nEjecución completada.')

SyntaxError: unterminated string literal (detected at line 47) (ipython-input-2015203786.py, line 47)